## Example Numerical Simulation through all 4 Melt-Freeze Stages

This notebook walks you through how to set up and run the numerical solution through the series of:
- Melting
- Freezing
- Dissolving (after antifreeze injection)
- Refreezing (slush)

Compare figure to that in the supplementary material of our manuscript (Fig. S1) (https://www.cambridge.org/core/journals/annals-of-glaciology/article/avoiding-slush-for-hotpoint-drilling-of-glacier-boreholes/6FDD0BE8243F1C625E0FAD398E8B5734).

In [ ]:
### Import libraries ###

# Basic libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

from scipy.optimize import minimize

# If the cylindricalstefan library is not in your PYTHONPATH, you will not be able to load those functions
# Check and update here if necessary
import sys
cs_dir = '/home/fenics/shared/'
if cs_dir not in sys.path:
    sys.path.append(cs_dir)
    
# Import functions and constants from the cylindricalstefan library
from cylindricalstefan.lib.instantaneous_mixing_model import *
from cylindricalstefan.lib.analytical_pure_solution import *
from cylindricalstefan.lib.constants import constantsHotPointDrill
const = constantsHotPointDrill()

In [ ]:
### Define optimization function ###

def C_opt(C,T_inf,data_dir):
    return abs(Tf_depression(C,data_dir=data_dir)-T_inf)

In [ ]:
### Run the simulations and plot ###

# Initialize figure
plt.figure()
ax1 = plt.subplot(311)
plt.tick_params(labelbottom=False,bottom=False)
ax1.set_ylabel('R$^*$')
ax2 = plt.subplot(312)
plt.tick_params(labelbottom=False,bottom=False)
ax2.set_ylabel('Dissolved')
ax3 = plt.subplot(313)
ax3.set_ylabel('Refrozen')
ax3.set_xlabel('t$^*$')

# Arrays for timing, equilibrium radius, and plotted color
TTs = np.array([.478,.478,.478,0.])
RRs = np.array([1.15,.6,.2,0.])
cs = ['indianred','gold','steelblue','k']

# Iterate through all simulations
for sim_i in range(len(RRs)):
    # Initialize the model
    mod = instantaneous_mixing_model()

    # Set the constants
    mod.T_inf = -20.          # bulk ice temperature
    mod.R_melt = 0.04         # melt-out radius
    mod.dt = 20.              # bigger time step to speed things up
    mod.t_final = 20000.      # end simulation time
    mod.Q_initialize = 1000.  # heat flux for melt out
    mod.Tf_last = 0.
    
    # Model setup
    mod.log_transform()
    mod.get_domain()

    # Get constants specific to this run
    C_final = minimize(C_opt,300,args=(mod.T_inf,'../data/'))['x'][0]  # final solution concentration
    Rfinal = RRs[sim_i]*mod.R_melt                                     # equilibrium radius
    mod.source_timing = TTs[sim_i]*mod.t0                              # when is the antifreeze injected?
    mod.source_mass_final = np.pi*Rfinal**2.*C_final                   # total amount of antifreeze

    # Continue with model setup
    mod.get_initial_conditions(data_dir='../data/')
    mod.save_times = mod.ts
    mod.get_boundary_conditions()
    
    # Run model
    mod.run(data_dir='../data/')

    # Important model output
    ts = mod.t0*mod.save_times[:len(mod.r_ice_result)]
    Rs = mod.r_ice_result[:,mod.ice_idx_wall]

    # Calculate dissolved and refrozen fraction from the results for model radius through time
    tinject = np.nan
    tfreeze = max(ts)+1.
    tdissolve = max(ts)+1.
    stage = 'freeze'
    Rlast = mod.R_melt
    for ti,R in enumerate(Rs):
        t = ts[ti]
        if abs(t-mod.source_timing*mod.t0) < 1e-5:
            Rinject = R
            tinject = t
        if R>Rlast and stage == 'freeze':
            Rfreeze = R
            tfreeze = t
            stage = 'dissolve'
        elif R<Rlast and stage =='dissolve':
            Rdissolve = R
            tdissolve = t
            stage = 'refreeze'
        Rlast = R
    if stage == 'freeze':
        if R<.001:
            pass
        else:
            Rdissolve = Rinject
            tdissolve = tinject
    elif stage == 'dissolve':
        Rdissolve = R
    Dissolved = (Rs**2. - Rfreeze**2.)/Rfreeze**2. + 1.
    Dissolved[ts<tfreeze] = 1.
    Dissolved[ts>tdissolve] = max(Dissolved)
    Refrozen = (Rdissolve**2. - Rs**2.)/Rdissolve**2.
    Refrozen[ts<tdissolve] = 0.

    # Plot the results
    ax1.plot(ts/mod.t0,Rs,c=cs[sim_i])
    ax2.plot(ts/mod.t0,Dissolved,c=cs[sim_i])
    ax3.plot(ts/mod.t0,Refrozen,c=cs[sim_i])